In [1]:
import pandas as pd
import numpy as np
import time
import psutil
import subprocess
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, Input

In [2]:
data= pd.read_csv(r"C:\Users\hsynt\Downloads\Selcuk Abi\Selcuk Abi\VeriSeti2.csv")

In [3]:
# GPU Kullanımını Ölçmek İçin Fonksiyon
def get_gpu_usage():
    try:
        result = subprocess.check_output(["nvidia-smi", "--query-gpu=utilization.gpu", "--format=csv,noheader,nounits"])
        gpu_usage = float(result.decode("utf-8").strip())
    except:
        gpu_usage = 0.0  # GPU yoksa veya erişilemezse 0 olarak döndür
    return gpu_usage

# CPU ve Bellek Kullanımını Ölçmek İçin Fonksiyon
def get_cpu_memory_usage():
    cpu_usage = psutil.cpu_percent(interval=1)
    memory_usage = psutil.virtual_memory().used / (1024 ** 3)  # GB cinsinden
    return cpu_usage, memory_usage

# Özellik ve Hedef Ayrımı
X = data.drop(columns=['Target_10', 'adjclose'])
y = data['Target_10']

# Zaman Serisi Formatına Getirme
def create_sequences(data, target, seq_length):
    sequences, targets = [], []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        targets.append(target[i + seq_length])
    return np.array(sequences), np.array(targets)

SEQ_LENGTH = 10
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_seq, y_seq = create_sequences(X_scaled, y.values, SEQ_LENGTH)

# PCA ve LDA Veri Hazırlığı
X_flat = X_seq.reshape(X_seq.shape[0], -1)  # Düzleştirilmiş veri

pca = PCA(n_components=10)
X_pca = pca.fit_transform(X_flat)

lda = LDA(n_components=1)
X_lda = lda.fit_transform(X_flat, y_seq)

# GRU Modeli Tanımlama
def build_gru_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        GRU(64, activation='relu', return_sequences=False),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Performans Hesaplama
results = []

for reduction, X_data in zip(
    ['No PCA/LDA', 'PCA', 'LDA'],
    [X_seq, X_pca, X_lda]
):
    if reduction == 'No PCA/LDA':
        X_data = X_data.reshape(X_data.shape[0], SEQ_LENGTH, -1)
    else:
        X_data = np.tile(X_data, (1, SEQ_LENGTH)).reshape(X_data.shape[0], SEQ_LENGTH, -1)

    # Modeli Oluştur
    gru_model = build_gru_model(X_data.shape[1:])
    
    # Modeli Eğit ve Kullanım Ölçümleri
    start_time = time.time()
    gru_model.fit(X_data, y_seq, epochs=3, batch_size=32, verbose=1)
    elapsed_time = time.time() - start_time

    # Kullanım Ölçümleri
    gpu_usage = get_gpu_usage()
    cpu_usage, memory_usage = get_cpu_memory_usage()

    results.append({
        'Reduction': reduction,
        'Model': 'GRU',
        'CPU Usage (%)': cpu_usage,
        'GPU Usage (%)': gpu_usage,
        'Memory Usage (GB)': memory_usage
    })

# Performans Sonuçlarını Görselleştirme
results_df = pd.DataFrame(results)

# CPU, GPU ve Bellek Kullanımı Tablosu
print("CPU, GPU ve Bellek Kullanımı Tablosu:")
print(results_df)

# Sonuçları Kaydetme
results_df.to_csv("gru_resource_usage.csv", index=False)
print("\nSonuçlar 'gru_resource_usage.csv' dosyasına kaydedildi.")

Epoch 1/3
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8317 - loss: 0.3894
Epoch 2/3
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8715 - loss: 0.3101
Epoch 3/3
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8763 - loss: 0.2966
Epoch 1/3
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.7671 - loss: 0.5099
Epoch 2/3
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8030 - loss: 0.4635
Epoch 3/3
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7973 - loss: 0.4637
Epoch 1/3
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8647 - loss: 0.3390
Epoch 2/3
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8679 - loss: 0.3101
Epoch 3/3
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8658 - loss: 0.3117
CPU, GPU ve Bellek Kullanımı Tablosu:
    Reduction Model  CPU Usage (%)  GPU Usage (%)  Memory Usage (GB)
0  No PCA/LDA   GRU            0.0            0.0          15.383133
1         PCA   GRU            0.2       